# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [3]:
df = pd.read_csv('data/Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [4]:
df.columns = df.columns.str.strip() ## we use the string method strip on the .columns attribute 

df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


In [5]:
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [6]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [7]:
df.isnull().sum() ## there is no missing data

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [8]:
df.set_index('Serial No.', drop = False) ## drop = False so the column isn't dropped

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
381,381,324,110,3,3.5,3.5,9.04,1,0.82
382,382,325,107,3,3.0,3.5,9.11,1,0.84
383,383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [9]:
print(df['GRE Score'].nunique())

print(df['CGPA'].nunique())

## I don't see how they can uniquely identify the data as the values repeat themselves in the dataframe

49
168


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [10]:
df.set_index(['GRE Score','CGPA'], inplace = True)

In [11]:
df # the combination between them does identify the dataframe uniquely

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...
324,9.04,381,110,3,3.5,3.5,1,0.82
325,9.11,382,107,3,3.0,3.5,1,0.84
330,9.45,383,116,4,5.0,4.5,1,0.91


In [12]:
df.reset_index(inplace=True)

In [13]:
df

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90
...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82
381,325,9.11,382,107,3,3.0,3.5,1,0.84
382,330,9.45,383,116,4,5.0,4.5,1,0.91
383,312,8.78,384,103,3,3.5,4.0,0,0.67


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [14]:
df[(df['CGPA'] > 9) & (df['Research'] == 1)]['Serial No.'].count() 


## filter the dataframe with both conditions, then count how many uniquely identified rows appear

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [15]:
df[(df['CGPA'] > 9) & (df['SOP'] < 3.5)]['Chance of Admit'].mean()

# same thing as above with the conditions. Select 'chance of admit' column then apply the mean

0.8019999999999999

In [16]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    return (df[col] - np.mean(df[col])) / np.std(df[col]) ## helped by jose, keep doing challenge below
    

In [17]:
standardize('CGPA')

0      1.750174
1     -0.992501
2      0.121191
3     -0.643433
4      1.234884
         ...   
380    0.736216
381    0.852571
382    1.417729
383    0.304036
384    1.766796
Name: CGPA, Length: 385, dtype: float64

In [18]:
df.head(2)

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [19]:
df['CGPA_std'] = standardize('CGPA')
df['GRE_std'] = standardize('GRE Score')
df['LOR_std'] = standardize('LOR')

df.head(3)

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517


We will generate the decision choice at random using the code below. Please run the cell.

In [20]:
# Libraries
from random import choices

In [21]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

df['decision_choice'] = decision_choice

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [22]:
df['deciding_column'] = df.lookup(df.index, df['decision_choice'])

df

/var/folders/v5/90hqz_xs4ys1hl5xzb98xyh40000gn/T/ipykernel_34324/3574504586.py:1: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  df['deciding_column'] = df.lookup(df.index, df['decision_choice'])


,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,decision_choice,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,GRE_std,1.755663
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std,-0.992501
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,GRE_std,0.456297
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,GRE_std,-0.236698
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,GRE_std,1.149292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,381,110,3,3.5,3.5,1,0.82,0.736216,0.629546,0.076840,LOR_std,0.076840
381,325,9.11,382,107,3,3.0,3.5,1,0.84,0.852571,0.716170,0.076840,CGPA_std,0.852571
382,330,9.45,383,116,4,5.0,4.5,1,0.91,1.417729,1.149292,1.193197,GRE_std,1.149292
383,312,8.78,384,103,3,3.5,4.0,0,0.67,0.304036,-0.409947,0.635019,CGPA_std,0.304036


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [23]:
def decider(row):
    if row['deciding_column'] > 0.8:
        val = 1
    else:
        val = 0
    return val


df['decision'] = df.apply(decider, axis=1)

In [24]:
df.head(3)

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,decision_choice,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,GRE_std,1.755663,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std,-0.992501,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,GRE_std,0.456297,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [25]:
df['decision'].sum()

93

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [26]:
df.columns = df.columns.str.replace(' ','_').str.replace('.','').str.lower()
df.head(3)

/var/folders/v5/90hqz_xs4ys1hl5xzb98xyh40000gn/T/ipykernel_34324/3372056209.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(' ','_').str.replace('.','').str.lower()


,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,decision_choice,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,GRE_std,1.755663,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std,-0.992501,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,GRE_std,0.456297,0


In [27]:
def boost(row):

    if row['university_rating'] >= 4:
        return row['gre_score'] + 10
    else:
        return row['gre_score']


df['gre_score'] = df.apply(boost, axis=1)

df.head(5)

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,decision_choice,deciding_column,decision
0,347,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,GRE_std,1.755663,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,CGPA_std,-0.992501,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,GRE_std,0.456297,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,GRE_std,-0.236698,0
4,340,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,GRE_std,1.149292,1


In [28]:
adjusted_gre = pd.cut(df['gre_score'], 4)

In [30]:
adjusted_gre.value_counts()

(305.0, 320.0]     150
(320.0, 335.0]      90
(335.0, 350.0]      75
(289.94, 305.0]     70
Name: gre_score, dtype: int64